<a href="https://colab.research.google.com/github/Goal48/NIFTY_Data_Analysis/blob/main/NIFTY50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/content/NIFTY 50-01-01-2007-to-31-12-2007.csv')

In [3]:
df.head()

,Date,Open,High,Low,Close,Shares Traded,Turnover (₹ Cr)
0,31-DEC-2007,6095.00,6167.75,6095.00,6138.60,85023208,4951.80
1,28-DEC-2007,6079.55,6098.60,6021.90,6079.70,87075023,5373.72
2,27-DEC-2007,6069.00,6110.85,6060.20,6081.50,150138294,9138.37
3,26-DEC-2007,5988.45,6085.25,5988.45,6070.75,90827443,5872.56
4,24-DEC-2007,5771.30,6001.05,5771.30,5985.10,100417081,6697.33


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             249 non-null    object 
 1   Open             249 non-null    float64
 2   High             249 non-null    float64
 3   Low              249 non-null    float64
 4   Close            249 non-null    float64
 5   Shares Traded    249 non-null    int64  
 6   Turnover (₹ Cr)  249 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 13.7+ KB


In [5]:
df.isnull().sum()

,0
Date,0
Open,0
High,0
Low,0
Close,0
Shares Traded,0
Turnover (₹ Cr),0
